In [ ]:
!pip install gensim

In [107]:
! pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [108]:
import PyPDF2
import nltk
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


Finding analogies

In [109]:
def extract_text_from_pdf(pdf_file):

    """Extracts text from a PDF file.

    Args:
        pdf_file: The path to the PDF file.

    Returns:
        A list of strings, each representing a page of the PDF.
    """

    text = []
    with open(pdf_file, 'rb') as pdf_reader:
        reader = PyPDF2.PdfReader(pdf_reader)
        for page in reader.pages:
            text.append(page.extract_text())
    return text

In [133]:
import nltk
nltk.download('punkt')

def chunk_text(text, chunk_size=300):
    """Chunks a text into smaller segments based on sentence boundaries.

    Args:
        text: The text to chunk.
        chunk_size: The desired maximum size of each chunk.

    Returns:
        A list of strings, each representing a chunk of the text.
    """

   # Join the list of strings into a single string
    text = ' '.join(text)

    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_chunk_size = 0

    for sentence in sentences:
        num_words = len(sentence.split())
        if current_chunk_size + num_words <= chunk_size:
            current_chunk.append(sentence)
            current_chunk_size += num_words
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_chunk_size = num_words

    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [111]:
def calculate_average_vector(text, wv):
    """Calculates the average Word2Vec vector for a text.

    Args:
        text: The text to process.
        wv: The Word2Vec model.

    Returns:
        The average Word2Vec vector for the text.
    """

    sent_vectors = []
    for sentence in text.split('.'):
        sent_vector = np.zeros((300,))
        count = 0
        for word in sentence.split():
            if word in wv:
                count += 1
                sent_vector += wv[word]
        if count > 0:
            sent_vectors.append(sent_vector / count)

    if sent_vectors:
        return sum(sent_vectors) / len(sent_vectors)
    else:
        return None

In [141]:

def find_similar_paragraphs(query, chunk_vectors, top_n=3):
    """Finds the most similar paragraphs to a query.

    Args:
        query: The query text.
        paragraph_vectors: A list of average Word2Vec vectors representing the paragraphs.
        top_n: The number of most similar paragraphs to return.

    Returns:
        A list of the most similar paragraphs.
    """

    query_vector = calculate_average_vector(query, wv)
    similarities = cosine_similarity([query_vector], chunk_vectors)[0]
    top_indices = similarities.argsort()[-top_n:][::-1]
    return [chunks[i] for i in top_indices]

In [114]:
# Load the PDF file and extract the text

pdf_file = '/content/sample_data/Improvingaccuracyofpretrainedwordembeddingsforsentimentanalysis.pdf'
text = extract_text_from_pdf(pdf_file)

In [117]:
print(len(text))
print(text[0])

15
Improving the Accuracy of Pre-trained Word Embeddings for 
Sentiment Analysis 
 
Seyed Mahdi Rezaeinia a,b, Ali Ghodsi  a, Rouhollah Rahmani  b 
a Department of Statistics and Actuarial Science, University of Waterloo , Waterloo,  Canada   
b Network Science and Technology Department, University of Tehran , Tehran, Iran  
 
 
Abstract  
Sentiment analysis is one of the well -known tasks and fast growing research areas in natural language 
processing (NLP) and text classifications. This technique has become an essential part of a wide range of 
applications  including politics,  business , advertising and marketing.  There are various techniques  for 
sentiment analysis, but r ecently  word embeddings methods  have  been widely used in sentiment 
classification tasks.  Word2Vec and GloV e are currently among the most accurate and usable word 
embedding methods which can convert words into meaningful vectors . However, these methods ignore 
sentiment information of texts  and need  a 

In [158]:
len(text[0])

2934

In [127]:
# Assuming 'text' is a list of strings where each string represents a sentence or paragraph
# and you want to count the total number of words in all strings within the list:
total_words = sum(len(sentence.split()) for sentence in text)

# Print the total number of words
print(total_words)

4894


In [134]:
# Chunk the text into smaller segments

chunks = chunk_text(text , 300)

In [135]:
len(chunks)

18

In [136]:
# Calculate average Word2Vec vectors for each chunk

chunk_vectors = [calculate_average_vector(chunk, wv) for chunk in chunks]

In [137]:
print(len(chunk_vectors))
print(chunk_vectors[0])

18
[ 0.02500691  0.03336662  0.04037175  0.08060622 -0.07895257  0.01929872
  0.00316947 -0.07053979  0.07782849  0.05283274 -0.03058976 -0.09124789
  0.02250694  0.04465533 -0.09763794  0.0581762   0.01454134  0.06499389
 -0.00192077 -0.08878534 -0.03924489  0.02250768 -0.02983831  0.03267289
 -0.01264794 -0.03057744 -0.11700708  0.03886558  0.00092304 -0.00326418
 -0.03715216 -0.04347403 -0.00144736 -0.02838854  0.01728254  0.0469247
 -0.05205797  0.04318216  0.09192727  0.00567166  0.08865903  0.00851114
  0.00708952  0.06012082  0.0024702  -0.04440909 -0.06786914 -0.00424986
 -0.01541388  0.05263813 -0.03905487  0.03958233 -0.05106843 -0.06978613
  0.00178676  0.07512456 -0.04147037 -0.08982822  0.028208   -0.06954416
 -0.04710114  0.01040004 -0.09551124 -0.07041034 -0.04339964  0.0108907
 -0.02542109  0.1044823   0.00659308  0.0269096  -0.00834378 -0.02253295
  0.09255501  0.01054245 -0.02575177 -0.06439972  0.08203788  0.04434809
  0.03397023  0.08976536  0.00273166 -0.05336929  

In [164]:
# Example usage:

query = " accuracy of the Word2vec and Glove ?"
similar_chunks = find_similar_paragraphs(query, chunk_vectors, top_n=2)


In [165]:
len(similar_chunks)

2

In [170]:
similar_chunks[0]

'[15] have used pre- trained Word2Vec as  a word embedding representation for recommending Idioms \nin essay w riting . As a result, i ncreasing the accuracy of pre- trained word embedding is very important and plays a vital role \nin sentiment classification methods. Zhang and Wallace  [10] combined pre- trained Word2Vec and GloVe \nvectors in their deep learning model, but the accuracies were  decreased. Some results are shown in       \ntable 1. Table1: The results of the combination of pre-trained Word2Vec and Glove on three datasets  [10] \nDataset  Reduced accuracy rate (%) \nMovie reviews ( MR) -0.22 %  \nStanford sentiment treebank  (SST)  -1.1%  \nTREC  -0.17%  \n \n \nAccording to table1, combination of pre- trained Word2Vec and Glove decreased the ac curacy  of text and \nsentiment classification on some datasets. Also, Kamkarhaghighi  and Makrehchi  [16] have proposed an \nalgorithm for increasing the accuracy of pre- trained Word2Vec and Glove. Their algorithm was tested o

In [171]:
similar_chunks[1]

'Figure  4: The accuracy  (%) comparisons of three methods on four deep learning models for CR dataset  \n \n 7777.57878.57979.580\nModel 1 Model 2 Model 3 Model 4The MR dataset\nGlove Word2Vec   IWV\n7979.58080.58181.58282.58383.5\nModel 1 Model 2 Model 3 Model 4The CR dataset\nGlove Word2Vec   IWV\nAccuracy (%)  \n Accuracy (%)   Figure 4 indicates that the IWV method generally has performed better than other per -trained word \nembeddings for sentiment analysis of CR dataset. The CR is an unbalanced dataset w hich contains 2397 \npositive and 1406 negative reviews. As can be seen, pre- trained Word2vec embedding is almost more \naccurate than pre- trained Glove embedding, however it is reverse in the model 2. The IWV provides  \nabsolute accuracy improvement s of 0.7%, 0.4%, 1.1% and 0.2% for model 1, model 2, model 3 and model \n4, respectively . 5. Conclusion  \n \nIn this paper, we proposed a new method to improve the accuracy of well -known pre- trained word \nembeddings  for se

Improvement of the solution by doing optimization